# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [16]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
weather_data = pd.read_csv ('weather_data.csv')
del weather_data["Unnamed: 0"]
weather_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Albany,42.6001,-73.9662,21.45,57,4,3.00,US,1644090484
1,Elizabeth City,36.2946,-76.2511,43.00,48,75,1.99,US,1644090176
2,Cayenne,4.9333,-52.3333,84.24,81,75,5.99,GF,1644090636
3,Cockburn Town,21.4612,-71.1419,80.56,54,40,11.50,TC,1644090115
4,Port Alfred,-33.5906,26.8910,70.48,83,93,25.52,ZA,1644090085
...,...,...,...,...,...,...,...,...,...
566,Half Moon Bay,37.4636,-122.4286,64.58,57,0,4.61,US,1644090830
567,Bonfim,3.0833,-59.9500,90.73,44,100,5.48,BR,1644090830
568,Bilibino,68.0546,166.4372,-15.63,93,15,4.59,RU,1644090831
569,Palana,59.1167,159.9667,3.07,86,100,7.67,RU,1644090831


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [6]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [47]:
# Store latitude and longitude in locations
location = weather_data[["Lat", "Lng"]]
humidity = weather_data['Humidity']

In [48]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(location, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# # Customize the size of the figure
# figure_layout = {
#     'width': '400px',
#     'height': '300px',
#     'border': '1px solid black',
#     'padding': '1px',
#     'margin': '0 auto 0 auto'
# }
# fig = gmaps.figure(layout=figure_layout)
fig
#ask gretel why there are multiple earths

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [10]:
# Fill NaN values
weather_data = weather_data.dropna()
weather_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Albany,42.6001,-73.9662,21.45,57,4,3.00,US,1644090484
1,Elizabeth City,36.2946,-76.2511,43.00,48,75,1.99,US,1644090176
2,Cayenne,4.9333,-52.3333,84.24,81,75,5.99,GF,1644090636
3,Cockburn Town,21.4612,-71.1419,80.56,54,40,11.50,TC,1644090115
4,Port Alfred,-33.5906,26.8910,70.48,83,93,25.52,ZA,1644090085
...,...,...,...,...,...,...,...,...,...
566,Half Moon Bay,37.4636,-122.4286,64.58,57,0,4.61,US,1644090830
567,Bonfim,3.0833,-59.9500,90.73,44,100,5.48,BR,1644090830
568,Bilibino,68.0546,166.4372,-15.63,93,15,4.59,RU,1644090831
569,Palana,59.1167,159.9667,3.07,86,100,7.67,RU,1644090831


In [15]:
weather_data.loc[weather_data["Max Temp"] <80]
weather_data_new = weather_data.loc[((weather_data['Max Temp'] <= 80) & (weather_data['Max Temp'] >= 70)) & ((weather_data['Wind Speed'] < 10) & (weather_data['Cloudiness'] == 0))]
weather_data_new
hotel_df = weather_data_new
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
8,Adrar,20.5022,-10.0711,71.10,7,0,9.08,MR,1644090638
56,Cape Town,-33.9258,18.4232,72.88,60,0,1.99,ZA,1644090279
57,Hilo,19.7297,-155.0900,77.11,74,0,0.00,US,1644090573
78,Chabahar,25.2919,60.6430,71.44,73,0,5.59,IR,1644090656
186,Mount Isa,-20.7333,139.5000,74.97,41,0,8.05,AU,1644090223
214,Atar,20.5169,-13.0499,77.43,9,0,6.91,MR,1644090002
276,Lompoc,34.6391,-120.4579,71.85,39,0,3.00,US,1644090725
310,Wamba,8.9333,8.6000,74.89,19,0,3.36,NG,1644090502
327,Najran,17.4924,44.1277,70.45,56,0,3.47,SA,1644090152
351,Sur,22.5667,59.5289,72.03,54,0,7.02,OM,1644090751


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [32]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
8,Adrar,20.5022,-10.0711,71.10,7,0,9.08,MR,1644090638
56,Cape Town,-33.9258,18.4232,72.88,60,0,1.99,ZA,1644090279
57,Hilo,19.7297,-155.0900,77.11,74,0,0.00,US,1644090573
78,Chabahar,25.2919,60.6430,71.44,73,0,5.59,IR,1644090656
186,Mount Isa,-20.7333,139.5000,74.97,41,0,8.05,AU,1644090223
214,Atar,20.5169,-13.0499,77.43,9,0,6.91,MR,1644090002
276,Lompoc,34.6391,-120.4579,71.85,39,0,3.00,US,1644090725
310,Wamba,8.9333,8.6000,74.89,19,0,3.36,NG,1644090502
327,Najran,17.4924,44.1277,70.45,56,0,3.47,SA,1644090152
351,Sur,22.5667,59.5289,72.03,54,0,7.02,OM,1644090751


In [43]:
hotel_names = []
lat = hotel_df['Lat']
lng = hotel_df['Lng']
lat_lngs = zip(lat,lng)

for lat_lng in lat_lngs:

    target_coordinates = f"{lat_lng[0]},{lat_lng[1]}"
    target_radius = 5000
    target_type = "hotel"


    params = {
        "location": target_coordinates,
        "radius": target_radius,
        "types": target_type,
        "key": g_key
    }

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    response = requests.get(base_url, params=params)
    
   
    places_data = response.json()

    print(json.dumps(places_data, indent=4, sort_keys=True))

    try:
        hotel_names.append(places_data["results"][1]["name"])
    except:
        print(f"No hotel found for {lat_lng}.")
        hotel_names.append("")

{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}
No hotel found for (20.5022, -10.0711).
{
    "html_attributions": [],
    "next_page_token": "Aap_uEAVMLMwJFCMYjG25Tnz8nKUzEzalaw2janodi0Piiv3wN63QV4TxS18es7NlMjnwC62dHjars6wdENIksIdj2RF69RITY9qHlB_XlM8Jq5fGHw6-SctdKeikiduhCD7ErVKUGvL3Sb9FapahpEzmYnCsOITjS1-cQhzxECYZd-Au4jJcFEgfRAKOcQccp2DRoYX4WI3M8sXnE30Ry7UrR5vdYuoFHDdeBsD0dpUJVD8PjrykQZIGnKqfEOjVv-fLyRVsbfyhn7LhN10VBmdJVti4HGi-vA8NBjFRSY3L5UN2aWwVTm-jLadE7ZNhl_VDEmkzA1sxLimyKd6c2-QlUdBOUiv-iXlVDXigc_Vo_28fPOWOjHcwKEC_kmMbnoPLODWe6HJfSLUDtvWJeZ0Zd_dtpdmz-spSm6aBI-PYC7gttG__HHzsw",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -33.9248685,
                    "lng": 18.4240553
                },
                "viewport": {
                    "northeast": {
                        "lat": -33.47127004603382,
                        "lng": 19.00466996799566
                    },
           

In [44]:
hotel_names

['',
 'Southern Sun Waterfront Cape Town',
 'Hilo Hawaiian Hotel',
 'Iran air',
 'ibis Styles Mt Isa Verona',
 'Oumou Elghoura',
 'Holiday Inn Express Lompoc, an IHG Hotel',
 'Goshen and Gold Schools',
 'مركز صحي البلد',
 'Sur Plaza Hotel',
 'BNDA SAN']

In [57]:
hotel_df["Hotel Name"] = hotel_names
print("There was no hotel located within 5000m of Adrar.")
hotel_df


There was no hotel located within 5000m of Adrar.


C:\Users\marqu\AppData\Local\Temp/ipykernel_26284/4052694212.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = hotel_names


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
8,Adrar,20.5022,-10.0711,71.10,7,0,9.08,MR,1644090638,
56,Cape Town,-33.9258,18.4232,72.88,60,0,1.99,ZA,1644090279,Southern Sun Waterfront Cape Town
57,Hilo,19.7297,-155.0900,77.11,74,0,0.00,US,1644090573,Hilo Hawaiian Hotel
78,Chabahar,25.2919,60.6430,71.44,73,0,5.59,IR,1644090656,Iran air
186,Mount Isa,-20.7333,139.5000,74.97,41,0,8.05,AU,1644090223,ibis Styles Mt Isa Verona
214,Atar,20.5169,-13.0499,77.43,9,0,6.91,MR,1644090002,Oumou Elghoura
276,Lompoc,34.6391,-120.4579,71.85,39,0,3.00,US,1644090725,"Holiday Inn Express Lompoc, an IHG Hotel"
310,Wamba,8.9333,8.6000,74.89,19,0,3.36,NG,1644090502,Goshen and Gold Schools
327,Najran,17.4924,44.1277,70.45,56,0,3.47,SA,1644090152,مركز صحي البلد
351,Sur,22.5667,59.5289,72.03,54,0,7.02,OM,1644090751,Sur Plaza Hotel


In [52]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [55]:
# Add marker layer on top of heat map
fig = gmaps.figure()

#create markers
markers = gmaps.marker_layer(locations)
#add markers
fig.add_layer(markers)
# Add heat layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))